In [56]:
# !pip install numpy 
# !pip install pandas 
# !pip install transformers 
# !pip3 install sentencepiece
# !pip3 install torch torchvision torchaudio
# !pip3 install scikit-learn
# !pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 4.8 MB/s eta 0:00:00a 0:00:01


In [23]:
import numpy as np
import pandas as pd
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [2]:
train_data = np.load("data/SP-train.npy", allow_pickle=True)
test_data = np.load("data/SP_eval_data_for_practice.npy", allow_pickle = True)

In [3]:
def get_data_lists(data):
    questions = []
    choices = []
    labels = []
    for example in data:
        # print(example.keys())
        questions.append(example['question'])
        choices.append(example['choice_list'])
        labels.append(example['label'])
    return questions, choices, labels
   

In [17]:
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)


In [18]:
train_questions, train_choices, train_labels = get_data_lists(train_data)
val_questions, val_choices, val_labels = get_data_lists(val_data)



In [19]:
print(train_data.shape, test_data.shape)
train_data[1]

(456,) (120,)


{'id': 'SP-9_SR',
 'question': 'Despite sharing the same parents, the 22nd and 24th presidents of the United States of America were not related. How is this even conceivable?',
 'answer': 'They were the same man.',
 'distractor1': 'Their parents got divorced.',
 'distractor2': 'The 24th president has better performance during tenure.',
 'distractor(unsure)': 'None of above.',
 'label': 2,
 'choice_list': ['The 24th president has better performance during tenure.',
  'Their parents got divorced.',
  'They were the same man.',
  'None of above.'],
 'choice_order': [2, 1, 0, 3]}

In [20]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-small')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# https://github.com/patil-suraj/exploring-T5/blob/master/t5_fine_tuning.ipynb

In [9]:
class Brain_Teaser(Dataset):
  def __init__(self, tokenizer, questions, choices, labels, max_len=512):
    self.questions = questions
    self.choices = choices
    self.labels = labels

    self.max_len = max_len
    self.tokenizer = tokenizer
    self.inputs = []
    self.targets = []

    self._buil_examples()
  
  def __len__(self):
    return len(self.inputs)
  
  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    # target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    # target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze
    return {"input_ids": source_ids, "attention_mask": src_mask, "labels" : self.targets[index]}
    # return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}
  

  def _buil_examples(self):
      for id in range(len(self.questions)):
        input = "Question : " + self.questions[id] + ' ' + "Options:" + ' ' + ' 0 '+ self.choices[id][0] + '1 '+ self.choices[id][1] + ' 2 '+ self.choices[id][2] + ' 3 '+ self.choices[id][3]
        label = self.labels[id]
        tokenized_inputs = self.tokenizer.batch_encode_plus(
            [input], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt"
        )
        # tokenized_targets = self.tokenizer.batch_encode_plus(
        #     [label], max_length=1, pad_to_max_length=True, return_tensors="pt"
        # )
        

        self.inputs.append(tokenized_inputs)
        self.targets.append(torch.tensor(label))
     

In [21]:
train_dataset = Brain_Teaser(tokenizer, train_questions, train_choices, train_labels)
val_dataset = Brain_Teaser(tokenizer, val_questions, val_choices, val_labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/nalishjain/Acad Sem 6/NLP-Project/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [24]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)


In [27]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',

)
    # output_dir="model_2c",
    # evaluation_strategy="epoch",
    # learning_rate=2e-5,
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=16,
    # weight_decay=0.01,
    # num_train_epochs=2,
    # predict_with_generate=True,
    # fp16=True,
    # logging_dir = './logs',
    # logging_strategy = 'epoch'
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

  0%|          | 0/1368 [00:00<?, ?it/s]

ValueError: not enough values to unpack (expected 2, got 1)

In [44]:
data = dataset[1]
print(tokenizer.decode(data['source_ids']))
print(data['label'])

Question : The six daughters of Mr. and Mrs. Mustard each have one brother. However, the family only consists of nine people; how is that possible? Options: 0 Some brothers were not loved by family and moved away.1 Some daughters get married and have their own family. 2 Each daughter shares the same brother. 3 None of above.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [27]:
dataset[0]['target_ids']

KeyError: 'target_ids'